In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import mne
from zipfile import ZipFile

In [3]:
import torch
import torch.nn as nn
import torcheeg as te

In [4]:
                        
                        
                                        torch.__version__
torch.cuda.is_available()

True

In [5]:
# !nvidia-smi

### Loading data

In [6]:
data_path = '../data/THINGS-EEG2/'

In [7]:
image_metadata_path = os.path.join(data_path, 'image_set/image_metadata.npy')
eeg_dir_train = os.path.join(data_path, 'preprocessed_data', 'sub-' + '01', 'preprocessed_eeg_' + 'training' + '.npy')
eeg_dir_test = os.path.join(data_path, 'preprocessed_data', 'sub-' + '01', 'preprocessed_eeg_' + 'test' + '.npy')

image_metadata = np.load(image_metadata_path, allow_pickle=True).item()
eeg_data_train = np.load(eeg_dir_train, allow_pickle=True).item()
eeg_data_test = np.load(eeg_dir_test, allow_pickle=True).item()

In [8]:
print(image_metadata.keys())
print(image_metadata['train_img_concepts'][:5])
print(image_metadata['train_img_concepts_THINGS'][:5])
print(image_metadata['train_img_files'][:5])

dict_keys(['test_img_concepts', 'test_img_concepts_THINGS', 'test_img_files', 'train_img_files', 'train_img_concepts', 'train_img_concepts_THINGS'])
['00001_aardvark', '00001_aardvark', '00001_aardvark', '00001_aardvark', '00001_aardvark']
['00001_aardvark', '00001_aardvark', '00001_aardvark', '00001_aardvark', '00001_aardvark']
['aardvark_01b.jpg', 'aardvark_02s.jpg', 'aardvark_03s.jpg', 'aardvark_04s.jpg', 'aardvark_05s.jpg']


In [9]:
# Get list of files names in zip
def get_labels(partition):
    path_tmp = os.path.join(data_path, 'image_set', f'{partition}_images.zip')

    with ZipFile(path_tmp, 'r') as zipObj:
        # Get list of files names in zip
        files_list_tmp = zipObj.namelist()

    files_list_tmp.sort()
    labels_tmp = ['_'.join(elem.split('/')[2].split('_')[: -1]) for elem in files_list_tmp if elem.endswith('/') == False]

    labels_tmp = np.repeat(labels_tmp, 4 if partition == 'training' else 80)

    return labels_tmp

In [10]:
labels_train = get_labels('training')
labels_test = get_labels('test')

print(len(labels_train))
print(len(labels_test))

66160
16000


In [11]:
labels_test

array(['aircraft_carrier', 'aircraft_carrier', 'aircraft_carrier', ...,
       'wok', 'wok', 'wok'], dtype='<U16')

In [12]:
eeg_data_train_reshaped = eeg_data_train['preprocessed_eeg_data'].reshape((66160, 17, 100))
eeg_data_test_reshaped = eeg_data_test['preprocessed_eeg_data'].reshape((16000, 17, 100))

print(eeg_data_train_reshaped.shape)
print(eeg_data_test_reshaped.shape)

(66160, 17, 100)
(16000, 17, 100)


### Classification

In [13]:
from torcheeg import models

In [14]:
model = models.EEGNet(chunk_size=100, num_electrodes=17, num_classes=200)

#### Versione 1

In [92]:
from torcheeg import datasets
from torcheeg import transforms
# Version 1.
# Mock 100 EEG samples. Each EEG signal contains a signal of length 1 s at a sampling rate of 128 sampled by 32 electrodes.
X = eeg_data_test_reshaped

# Mock 100 labels, denoting valence and arousal of subjects during EEG recording.
y = {'prova': labels_test}

dataset = datasets.NumpyDataset(X=X,y=y)

# dataset = datasets.NumpyDataset(X=X,
#                        y=y,
#                     #    io_path=io_path,
#                     #    offline_transform=transforms.Compose(
#                     #        [transforms.BandDifferentialEntropy()]),
#                        online_transform=transforms.ToTensor(),
#                        label_transform=transforms.Compose([
#                            transforms.Select('prova'),
#                            transforms.Binary(5.0),
#                        ]),
#                        num_worker=2,
#                        num_samples_per_worker=50)
print(dataset[0])
# EEG signal (torch.Tensor[32, 4]),
# coresponding baseline signal (torch.Tensor[32, 4]),
# label (int)

The current io_size is not enough, and double the LMDB map size to 20971520 automatically.


The current io_size is not enough, and double the LMDB map size to 41943040 automatically.


[NUMPY]:   8%|▊         | 13/160 [00:09<02:00,  1.22it/s]

The current io_size is not enough, and double the LMDB map size to 83886080 automatically.


The current io_size is not enough, and double the LMDB map size to 167772160 automatically.


The current io_size is not enough, and double the LMDB map size to 335544320 automatically.


[NUMPY]: 100%|██████████| 160/160 [02:35<00:00,  1.03it/s]

Please wait for the writing process to complete...
(array([[-0.00884811,  0.08756441, -0.52737595, ..., -2.16473542,
        -1.3355974 , -0.72026964],
       [ 0.06150865,  0.51575416,  0.32667771, ...,  0.970259  ,
         0.51827952,  0.37765688],
       [ 0.17327892,  0.2225759 ,  0.26673386, ...,  0.26644941,
         0.24309482,  0.20390374],
       ...,
       [ 0.45961279,  0.20282258, -0.27539875, ..., -1.31372021,
        -1.06018159, -0.82630036],
       [ 0.46353537,  0.34022384,  0.20510332, ..., -0.9093181 ,
         0.64570811,  0.88260709],
       [ 0.42694804, -0.01804091, -0.18026478, ..., -0.7314609 ,
        -0.01432714,  0.26942955]]), {'clip_id': '0_0', 'prova': 'aircraft_carrier'})


In [93]:
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}

# Generators
training_set = dataset
training_generator = torch.utils.data.DataLoader(training_set, **params)

In [21]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Using cuda device


In [23]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [24]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_generator, model, loss_fn, optimizer)
    test(training_generator, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

#### Versione 2

In [101]:
# Version 2. Define dataloader + splitting
# dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.

class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, x, labels):
        'Initialization'
        self.labels = labels
        self.x = x

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.labels)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        X = self.x[index]
        y = self.labels[index]

        return X, y

In [103]:
# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}

# Generators
training_set = Dataset(eeg_data_test_reshaped, labels_test)
training_generator = torch.utils.data.DataLoader(training_set, **params)

# Dataset(x, labels)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
loss_fn = nn.CrossEntropyLoss()
batch_size = 64

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_idx, batch in enumerate(dataloader):
        X = batch[0].to(device)
        y = batch[1].to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            loss, current = loss.item(), batch_idx * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def valid(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            X = batch[0].to(device)
            y = batch[1].to(device)

            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {val_loss:>8f} \n")

In [ ]:
for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):

    model = CNN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    epochs = 50
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_loader, model, loss_fn, optimizer)
        valid(val_loader, model, loss_fn)
    print("Done!")

#### Versione 3

In [15]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class Data(Dataset):
  def __init__(self, X_train, y_train):
    # need to convert float64 to float32 else 
    # will get the following error
    # RuntimeError: expected scalar type Double but found Float
    self.X = torch.from_numpy(X_train.astype(np.float32))
    # need to convert float64 to Long else 
    # will get the following error
    # RuntimeError: expected scalar type Long but found Float
    self.y = torch.from_numpy(y_train).type(torch.LongTensor)
    self.len = self.X.shape[0]
  
  def __getitem__(self, index):
    return self.X[index], self.y[index]
  def __len__(self):
    return self.len

In [21]:
labels_test_int = np.repeat(np.array(range(200)), 80)

data = Data(eeg_data_test_reshaped, labels_test_int)

In [22]:
batch_size = 64
trainloader = DataLoader(data, batch_size=batch_size, 
                         shuffle=True)

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.9)

In [25]:
epochs = 10
for epoch in range(epochs):
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    # print('fds')
    inputs, labels = data
    # set optimizer to zero grad to remove previous epoch gradients
    optimizer.zero_grad()
    # print(inputs.shape)
    # forward propagation
    inputs = inputs.view(batch_size, 1, 17, 100) # N, C, H, W (batch size, channels, height, width)
    # print(inputs.shape)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    # backward propagation
    loss.backward()
    # optimize
    optimizer.step()
    running_loss += loss.item()
    # print(i)
  # display statistics
  print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.5f}')

[1,   250] loss: 0.65252
[2,   250] loss: 0.62010
[3,   250] loss: 0.59741
[4,   250] loss: 0.58129
[5,   250] loss: 0.57021
[6,   250] loss: 0.56229
[7,   250] loss: 0.55468


KeyboardInterrupt: 